In [3]:
import json
import pandas as pd
import colorsys
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from os import listdir
from PIL import Image as PImage

from utils.image_utils import *
from utils.data_utils import GaussianClustering




In [ ]:
def rgb_to_saturation(c):
    r, g, b = c[0] / 255.0, c[1] / 255.0, c[2] / 255.0
    hsv = colorsys.rgb_to_hsv(r,g,b)
    return hsv[1]

In [ ]:
def get_main_colors_clusters(imgPath): 
    # get image and convert pixels into a dataframe
    pimg = PImage.open(imgPath).convert("RGB")
    pxs = get_pixels(pimg)
    pxs_df = pd.DataFrame(pxs, columns = ['R', 'G', 'B'])
    print("got pixels")
    

    # create clustering model
    model = GaussianClustering(n_clusters = 8) #get 8 colors first
    predicted = model.fit_predict(pxs_df[['R', 'G', 'B']])
    print("trained model")

    # post process clustering result
    ccounts = predicted['clusters'].value_counts()
    colors = []
    for clusterNum in ccounts.index[:5]:
        r,g,b = model.cluster_centers_[clusterNum]
        colors.append([round(r), round(g), round(b)])
    print('getting color info')

    sorted_colors = sorted(colors, key=rgb_to_saturation, reverse=True)

    return sorted_colors, pxs_df, colors, predicted


In [ ]:
sorted_colors, pxs_df, colors, predicted = get_main_colors_clusters('./tmp/past.png')

KeyboardInterrupt: 

In [ ]:

# plotting
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(pxs_df['R'], pxs_df['G'], pxs_df['B'], c=predicted, s=1, alpha=0.5)
ax.set_xlabel('R')
ax.set_ylabel('G')
ax.set_zlabel('B')
for idx, color in enumerate(colors):
    ax.scatter(color[0], color[1], color[2], c=[color], marker='o', s=100, label=f"Cluster {idx+1}")

ax.legend()
plt.show()


In [ ]:
def get_main_colors(imgPath,  color_similarity_threshold = 100):
    pimg =  PImage.open(imgPath).convert("RGB")
    pxs = get_pixels(pimg)

    main_colors = []
    def color_distance(c1, c2):
        d = 0
        for i in range(len(c1)):
            d += (c1[i]-c2[i])**2
        d = d**.5
        return d

    def color_unique(c):
        for existing_color in main_colors:
            dist = color_distance(existing_color, c) 
            # print(dist)
            if dist < color_similarity_threshold: 
                return False, existing_color
        
        return True, c

    for color in pxs:
        unique, color = color_unique(color)

        if unique:
            main_colors.append(color)

    main_colors = [list(c) for c in main_colors]
    sorted_colors = sorted(main_colors, key=rgb_to_saturation, reverse=True)
    return sorted_colors[:10]
